In [1]:
import pandas as pd
from langdetect import DetectorFactory, detect_langs
DetectorFactory.seed = 0

data = pd.read_csv("data/all_songs_data.csv", nrows= 100)

In [7]:
data.shape

(100, 12)

In [24]:
df_lang = pd.DataFrame(columns = ["en"])
for ii in data.index:
    try:
        for lang in detect_langs(data.loc[ii,"Lyrics"]):
            df_lang.loc[ii, lang.lang] = lang.prob
    except:
        pass
    

,en,pt,so,ro,de,id
0,0.999999,NaN,NaN,NaN,NaN,NaN
1,0.999998,NaN,NaN,NaN,NaN,NaN
2,0.999996,NaN,NaN,NaN,NaN,NaN
3,0.999998,NaN,NaN,NaN,NaN,NaN
4,0.999996,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
94,0.714286,NaN,NaN,NaN,NaN,0.285714
95,0.347326,NaN,NaN,NaN,0.652671,NaN
96,0.999996,NaN,NaN,NaN,NaN,NaN
97,0.999996,NaN,NaN,NaN,NaN,NaN


In [45]:
pd.DataFrame(lang_list).query()

,en,pt,so,ro,de,id
0,0.999999,NaN,NaN,NaN,NaN,NaN
1,0.999998,NaN,NaN,NaN,NaN,NaN
2,0.999996,NaN,NaN,NaN,NaN,NaN
3,0.999998,NaN,NaN,NaN,NaN,NaN
4,0.999996,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
99,0.347326,NaN,NaN,NaN,NaN,NaN
100,0.999996,NaN,NaN,NaN,NaN,NaN
101,0.999996,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN


: 